In [ ]:
# Definimos las variables de entrada de la Sankey
df_sankey_0 = df_pruebas_1[~df_pruebas_1['v1_prepost'].isin(['Ambos','Ninguno'])]
variables_sankey = ["v1_prepost","v2_Estado", "ACCE_EDO"]
variables_sankey_ = ["v1_prepost", "ACCE_EDO"]
valor_sankey = "version_count"

#Mapeo de los Nombres de variables 
mapeo_v1_prepost = {'Ambos':'Ambos',
                  'Post':'Post-Implantación',
                  'Ninguno':'Ninguno',
                  'Previo': 'Pre-Implantación'}

df_sankey_0["v1_prepost"] = df_sankey_0["v1_prepost"].map(mapeo_v1_prepost)


mapeo_v2_Estado = {'Aceptada':'Aceptada',
                  'No_evaluable':'No evaluable',
                  'Rechazada':'Rechazada'}

df_sankey_0["v2_Estado"] = df_sankey_0["v2_Estado"].map(mapeo_v2_Estado)


mapeo_acce_edo = {0:'0 - Sin Hacer',
                  1:'1 - Aceptada',
                  2:'2 - Aceptada con Obs.',
                  3:'3 - Rechazada'}

df_sankey_0["ACCE_EDO"] = df_sankey_0["ACCE_EDO"].map(mapeo_acce_edo)

# Definición de la Paleta de colores
paleta_colores = ['#4B8BBE','#306998','#FFE873','#FFD43B','#646464']
nodos_sankey = []
colorNumList = []
for catCol in variables_sankey:
    nodos_sankeyTemp =  list(set(df_sankey_0[catCol].values))
    colorNumList.append(len(nodos_sankeyTemp))
    nodos_sankey = nodos_sankey + nodos_sankeyTemp
    
# Definimos los nodos de la Sankey
nodos_sankey = list(dict.fromkeys(nodos_sankey))

# Definimos los colores en función de la jerarquia
colorList = []
for idx, colorNum in enumerate(colorNumList):
    colorList = colorList + [paleta_colores[idx]]*colorNum

#Creamos un dataframe para asignar el origen y destino de las diferentes ramas de la Sankey
for i in range(len(variables_sankey)-1):
    if i==0:
        df_origen_sankey = df_sankey_0[[variables_sankey[i],variables_sankey[i+1], valor_sankey]]
        df_origen_sankey.columns = ["fuente", "destino", "total"]
    else:
        df_temp_sankey = df_sankey_0[[variables_sankey[i],variables_sankey[i+1], valor_sankey]]
        df_temp_sankey.columns = ["fuente", "destino", "total"]
        df_origen_sankey = pd.concat([df_origen_sankey,df_temp_sankey])
    df_sankey = df_origen_sankey.groupby(["fuente", "destino"]).agg({"total":"sum"}).reset_index()

df_sankey["indice_fuente"] = df_sankey["fuente"].apply(lambda x: nodos_sankey.index(x))
df_sankey["indice_destino"] = df_sankey["destino"].apply(lambda x: nodos_sankey.index(x))

df_sankey[:10]

lista_colores = [
                    "rgba(31, 119, 180, 0.8)",
                    "rgba(255, 127, 14, 0.8)",
                    "rgba(44, 160, 44, 0.8)",
                    "rgba(214, 39, 40, 0.8)",
                    "rgba(148, 103, 189, 0.8)",
                    "rgba(140, 86, 75, 0.8)",
                    "rgba(227, 119, 194, 0.8)",
                    "rgba(127, 127, 127, 0.8)",
                    "rgba(188, 189, 34, 0.8)",
                    "rgba(23, 190, 207, 0.8)",
                    "rgba(31, 119, 180, 0.8)",
                    "rgba(255, 127, 14, 0.8)",
                    "rgba(44, 160, 44, 0.8)",
                    "rgba(214, 39, 40, 0.8)",
                    "rgba(148, 103, 189, 0.8)",
                    "rgba(140, 86, 75, 0.8)",
                    "rgba(227, 119, 194, 0.8)",
                    "rgba(127, 127, 127, 0.8)"]

df_sankey['colores'] = lista_colores[:len(df_sankey)]

#Definición de la estrctura básica de la Sankey
sankey = go.Figure(dict(
        type='sankey',
        node = dict(
          pad = 15,
          thickness = 20,
          line = dict(
            color = "black",
            width = 0.5
          ),
          label = nodos_sankey,
          color = colorList
        ),
        link = dict(
          source = df_sankey['indice_fuente'],
          target = df_sankey['indice_destino'],
          value = df_sankey['total'],
          color = df_sankey['colores']
        )
      ))

sankey.update_layout(margin = dict(t=50, l=70, r=20, b=70), width= 1200, height=400,
                              title = 'Flujo Pruebas de Accesibilidad a las Versiones de Aplicación', title_x=0.5, title_font=dict(color='rgba(0,171,203,1)'),
                              plot_bgcolor='rgba(0,171,203,0.2)')